- Average Annual Household Transportation Costs: Out-of-pocket expenses including tolls, parking, and transit fares

    - These costs do not include other vehicle operating and purchase costs.

    - Costs are annualized using factor of 320


In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '${:0,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# person data
person = pd.read_csv(output_path / 'agg/dash/person_cost.csv')
# household data
hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')

annual_factor = summary_config['weekday_to_annual']
# list of equity geographies
equity_geogs = summary_config['equity_geogs']
not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
df_person = person.copy()
df_person['is_rgc'] = "In RGC"
df_person.loc[df_person['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"
# included modes
# TODO: ask what this means
df_person = df_person[(df_person['mode'].isin(['SOV','HOV2','HOV3+']) & (df_person['dorp'] == 1)) 
                      | (~df_person['mode'].isin(['SOV','HOV2','HOV3+']))]

df_hh = hh.copy()
df_hh['is_rgc'] = "In RGC"
df_hh.loc[df_hh['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"

In [5]:
def annual_costs(geog, map=False):
    df = df_person.groupby(geog, as_index=False)['travcost_wt'].sum().set_index(geog)
    df['hhexpfac'] = df_hh.groupby(geog)['hhexpfac'].sum()

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )

    df.loc['Region'] = [df_person['travcost_wt'].sum(), df_hh['hhexpfac'].sum()]
    df['Average Annual Costs'] = (df['travcost_wt'] / df['hhexpfac']) * annual_factor




    return df[['Average Annual Costs']]

In [6]:
annual_costs('hh_county')

,Average Annual Costs
hh_county,
King,"$2,383"
Kitsap,"$2,837"
Outside Region,"$4,877"
Pierce,"$2,783"
Snohomish,"$2,953"
Region,"$2,597"


In [7]:
annual_costs('is_rgc')

,Average Annual Costs
is_rgc,
In RGC,$954
Not in RGC,"$2,772"
Region,"$2,597"


In [8]:
annual_costs('hh_rgc')

,Average Annual Costs
hh_rgc,
Auburn,"$1,868"
Bellevue,$985
Bothell Canyon Park,"$2,391"
Bremerton,"$1,164"
Burien,"$1,815"
Everett,"$1,096"
Federal Way,"$1,947"
Greater Downtown Kirkland,"$1,824"
Kent,"$1,682"


In [9]:
annual_costs('hh_rg_proposed')

,Average Annual Costs
hh_rg_proposed,
Cities and Towns,"$3,399"
Core,"$2,528"
HCT,"$2,751"
Metro,"$1,689"
Rural,"$4,387"
Urban Unincorporated,"$3,187"
Region,"$2,597"


#### Equity Focus Areas

In [10]:
df_person.columns

Index(['dorp', 'mode', 'hh_rg_proposed', 'hh_county', 'hh_rgc', 'hh_city',
       'hh_efa_dis', 'hh_efa_older', 'hh_efa_lep', 'hh_efa_pov200',
       'hh_efa_poc', 'hh_efa_youth', 'travcost_wt', 'is_rgc'],
      dtype='object')

In [11]:
annual_costs('hh_efa_poc', map=True)

,Average Annual Costs
hh_efa_poc,
Below Regional Average,"$2,811"
Above Regional Average,"$2,336"
Higher Share of Equity Population,"$2,354"
Region,"$2,597"


In [12]:
annual_costs('hh_efa_pov200', map=True)

,Average Annual Costs
hh_efa_pov200,
Below Regional Average,"$2,806"
Above Regional Average,"$2,410"
Higher Share of Equity Population,"$2,062"
Region,"$2,597"


In [13]:
annual_costs('hh_efa_lep', map=True)

,Average Annual Costs
hh_efa_lep,
Below Regional Average,"$2,653"
Above Regional Average,"$2,540"
Higher Share of Equity Population,"$2,457"
Region,"$2,597"


In [14]:
annual_costs('hh_efa_dis', map=True)

,Average Annual Costs
hh_efa_dis,
Below Regional Average,"$2,693"
Above Regional Average,"$2,603"
Higher Share of Equity Population,"$2,245"
Region,"$2,597"


In [15]:
annual_costs('hh_efa_older', map=True)

,Average Annual Costs
hh_efa_older,
Below Regional Average,"$2,463"
Above Regional Average,"$2,689"
Higher Share of Equity Population,"$2,887"
Region,"$2,597"


In [16]:
annual_costs('hh_efa_youth', map=True)

,Average Annual Costs
hh_efa_youth,
Below Regional Average,"$2,212"
Above Regional Average,"$2,984"
Higher Share of Equity Population,"$3,284"
Region,"$2,597"
